In [1]:
# import functions
import idaes
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)

from idaes.core import FlowsheetBlock
import idaes.logger as idaeslog
from pyomo.opt import TerminationCondition, SolverStatus
# Import the Generic Parameter Block
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)
# Import unit models from the model library
from idaes.generic_models.unit_models import Flash
# Import degrees of freedom tool
from idaes.core.util.model_statistics import degrees_of_freedom
# parmest (binary_param2)
from binary_param import cuadratic, linear, constant
# import pressure and plot functions
from binary_fitting_functions import calc_outlet_xP
from binary_fitting_functions import plot_sens_analysis

In [2]:
# from HFC32_emimTf2N_PR import configuration
from HFC32_emimTf2N_PR import configuration

In [3]:
#need to make excel csv for 125
data = pd.read_csv('R32_emimTf2N2.csv')
# print(data)

In [4]:
# 283
data_283= data[(data.temperature > 282) & (data.temperature< 284)]

# 298
data_298 = data[(data.temperature > 297) & (data.temperature< 299)]
data_298 = data_298.reset_index(drop=True)

# 323
data_323 = data[(data.temperature > 322) & (data.temperature< 324)]
data_323 = data_323.reset_index(drop=True)

# 323
data_348 = data[(data.temperature > 347) & (data.temperature< 349)]
data_348 = data_348.reset_index(drop=True)


In [5]:
m = ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False})
m.fs.properties = GenericParameterBlock(default=configuration)
m.fs.F101 = Flash(default={"property_package": m.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})
DOF_initial = degrees_of_freedom(m)
print('The initial DOF is {0}'.format(DOF_initial))

The initial DOF is 7


# Parameter estimation

In [6]:
print(data)

    temperature  pressure  x_R32  x_emimTf2N
0        283.15    100100  0.136       0.864
1        283.15    250300  0.306       0.694
2        283.15    399300  0.448       0.552
3        283.15    549300  0.570       0.430
4        283.15    700400  0.672       0.328
5        283.15    849400  0.786       0.214
6        298.15     99800  0.095       0.905
7        298.15    250200  0.218       0.782
8        298.15    399400  0.325       0.675
9        298.15    549400  0.417       0.583
10       298.15    699500  0.499       0.501
11       298.15    849500  0.570       0.430
12       298.15    999700  0.643       0.357
13       323.15    100300  0.055       0.945
14       323.15    249700  0.133       0.867
15       323.15    400100  0.200       0.800
16       323.15    549500  0.264       0.736
17       323.15    700300  0.320       0.680
18       323.15    850400  0.368       0.632
19       323.15   1000400  0.417       0.583
20       348.05     99800  0.034       0.966
21       3

In [7]:
cuadratic(data, configuration, 'R32', 'emimTf2N', "x_R32", "x_emimTf2N", 
    init_temp =  283.1, init_press =   399300 , init_x_c1 =    0.448, init_x_c2 = 0.552,
    init_kappa_2_1A = 8.8718, init_kappa_1_2A = 1.1017,
    init_kappa_2_1B = -15.8940, init_kappa_1_2B = -1.9784,
    init_kappa_2_1C = 6.8469053671, init_kappa_1_2C = 0.8358154970, 
    eps = 0.1, scaling_fac = 1e-7)

2021-08-17 12:42:57 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-17 12:42:58 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-17 12:42:59 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-17 12:43:00 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-17 12:43:01 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-17 12:43:02 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-17 12:43:03 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-17 12:43:04 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-17 12:43:05 [INFO] idaes

The SSE at the optimal solution is 4.386362

The values for the parameters are as follows:
fs.properties.PR_kappa_A[emimTf2N,R32] = 4.260420071146809
fs.properties.PR_kappa_A[R32,emimTf2N] = 0.4969906835940117
fs.properties.PR_kappa_B[emimTf2N,R32] = -7.359503058312215
fs.properties.PR_kappa_B[R32,emimTf2N] = -0.8976688948247294
fs.properties.PR_kappa_C[emimTf2N,R32] = 2.928793297605516
fs.properties.PR_kappa_C[R32,emimTf2N] = 0.35439612180112207
covariance_matrix                                         fs.properties.PR_kappa_A[emimTf2N,R32]  \
fs.properties.PR_kappa_A[emimTf2N,R32]                                0.021598   
fs.properties.PR_kappa_A[R32,emimTf2N]                               -0.016580   
fs.properties.PR_kappa_B[emimTf2N,R32]                               -0.060943   
fs.properties.PR_kappa_B[R32,emimTf2N]                                0.030149   
fs.properties.PR_kappa_C[emimTf2N,R32]                                0.039357   
fs.properties.PR_kappa_C[R32,emimTf2N] 

In [8]:
linear(data, configuration, 'R32', 'emimTf2N', "x_R32", "x_emimTf2N", 
    init_temp =  283.1, init_press =   399300 , init_x_c1 =    0.448, init_x_c2 = 0.552,
    init_kappa_2_1A = 1.2354, init_kappa_1_2A = 0.1695,
    init_kappa_2_1B = -1.387, init_kappa_1_2B = -0.2075, eps = 0.1, scaling_fac = 1e-9)

2021-08-17 12:43:58 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-17 12:44:00 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-17 12:44:02 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-17 12:44:04 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-08-17 12:44:05 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-08-17 12:44:07 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-17 12:44:08 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-17 12:44:10 [INFO] idaes.init.fs.state_block: Pro

In iteration 32, 27 Slacks too small, adjusting variable bounds
  33  2.3522944e-01 1.48e-02 1.43e+12  -1.0 3.66e+03    -  1.00e+00 9.90e-01w  1
  34  2.3746198e-01 7.74e-05 9.77e-03  -1.0 1.92e+02    -  1.00e+00 1.00e+00h  1
  35  1.0875231e-01 8.09e-02 1.12e+12  -5.7 1.31e+04    -  9.29e-01 1.00e+00f  1
  36  4.9780382e-02 1.75e-01 9.07e+10  -5.7 1.25e+04    -  9.19e-01 1.00e+00h  1
  37  4.5380048e-02 5.85e-02 4.17e+07  -5.7 3.21e+03    -  1.00e+00 1.00e+00h  1
  38  4.5092202e-02 8.99e-03 1.46e-02  -5.7 8.52e+02    -  1.00e+00 1.00e+00h  1
  39  4.5088099e-02 1.36e-04 4.09e-04  -5.7 1.15e+02    -  1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  40  4.5088098e-02 6.06e-08 5.96e-08  -5.7 2.16e+00    -  1.00e+00 1.00e+00h  1
  41  4.5088096e-02 3.66e-08 1.72e-07  -8.6 2.56e+00    -  1.00e+00 1.00e+00h  1
  42  4.5088096e-02 5.11e-12 5.82e-11  -8.6 9.13e-04    -  1.00e+00 1.00e+00h  1

Number of Iterations....: 42

              

c:\users\agarc\anaconda32\envs\kappa\lib\site-packages\pyomo\contrib\interior_point\interface.py:418: RuntimeWarning: divide by zero encountered in true_divide
  data = (duals_primals_lb/(primals - self._nlp.primals_lb()) +


In [9]:
constant(data, configuration, 'R32', 'emimTf2N', "x_R32", "x_emimTf2N", 
    init_temp =  283.1, init_press =   399300 , init_x_c1 =    0.448, init_x_c2 = 0.552,
    init_kappa_A_2_1 = -0.035, init_kappa_A_1_2 = -0.02, eps = 0.1, scaling_fac = 1e-9 , read=False)

2021-08-17 12:45:11 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-17 12:45:12 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-17 12:45:14 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-17 12:45:15 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-17 12:45:17 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-17 12:45:18 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-17 12:45:20 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-17 12:45:21 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-17 12:45:23 [INFO] idaes